In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
import os
from tqdm.notebook import tqdm

In [4]:
!unzip archive.zip

Archive:  archive.zip
replace images/images/train/angry/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
train_dir = "images/train"
test_dir = "images/validation"

In [6]:
def createDataFrame(dir):
  image_paths=[]
  labels = []
  for label in os.listdir(dir):
    for imagename in os.listdir(os.path.join(dir, label)):
      image_paths.append(os.path.join(dir, label, imagename))
      labels.append(label)
    print(label, "Completed")
  return image_paths, labels

In [7]:
train = pd.DataFrame()
train['image'], train['label'] = createDataFrame(train_dir)

disgust Completed
sad Completed
surprise Completed
angry Completed
neutral Completed
happy Completed
fear Completed


In [8]:
print(train)

                                image    label
0      images/train/disgust/20052.jpg  disgust
1      images/train/disgust/26984.jpg  disgust
2      images/train/disgust/29184.jpg  disgust
3       images/train/disgust/5667.jpg  disgust
4      images/train/disgust/22196.jpg  disgust
...                               ...      ...
28816     images/train/fear/14125.jpg     fear
28817     images/train/fear/18529.jpg     fear
28818      images/train/fear/1721.jpg     fear
28819     images/train/fear/29994.jpg     fear
28820      images/train/fear/3640.jpg     fear

[28821 rows x 2 columns]


In [9]:
test = pd.DataFrame()
test['image'], test['label'] = createDataFrame(test_dir)

disgust Completed
sad Completed
surprise Completed
angry Completed
neutral Completed
happy Completed
fear Completed


In [10]:
print(test)

                                    image    label
0     images/validation/disgust/14954.jpg  disgust
1       images/validation/disgust/807.jpg  disgust
2     images/validation/disgust/16264.jpg  disgust
3      images/validation/disgust/3101.jpg  disgust
4      images/validation/disgust/4737.jpg  disgust
...                                   ...      ...
7061      images/validation/fear/3472.jpg     fear
7062     images/validation/fear/17710.jpg     fear
7063      images/validation/fear/2592.jpg     fear
7064     images/validation/fear/25306.jpg     fear
7065      images/validation/fear/2988.jpg     fear

[7066 rows x 2 columns]


In [11]:
def extractFeatures(images):
  features=[]
  for i in tqdm(images):
    img = load_img(i, color_mode = "grayscale")
    img = np.array(img)
    features.append(img)
  features = np.array(features)
  features = features.reshape(len(features), 48, 48, 1)
  return features




In [12]:
train_features = extractFeatures(train['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

In [13]:
test_features = extractFeatures(test['image'])

  0%|          | 0/7066 [00:00<?, ?it/s]

In [14]:
x_train = train_features/255.0
x_test = test_features/255.0

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
le = LabelEncoder()

In [17]:
le.fit(train['label'])

LabelEncoder()

In [18]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [19]:
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [20]:
model = Sequential()
model.add(Conv2D(128, kernel_size=(3,3), activation="relu", input_shape = (48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())

# Fully Connected Layer
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(216, activation="relu"))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(7, activation="softmax"))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics = ["accuracy"])

In [23]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=50, validation_data = (x_test, y_test))

Epoch 1/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - accuracy: 0.5557 - loss: 1.1745 - val_accuracy: 0.5637 - val_loss: 1.1433
Epoch 2/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 21s 48ms/step - accuracy: 0.5668 - loss: 1.1478 - val_accuracy: 0.5699 - val_loss: 1.1476
Epoch 3/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - accuracy: 0.5655 - loss: 1.1435 - val_accuracy: 0.5795 - val_loss: 1.1109
Epoch 4/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - accuracy: 0.5772 - loss: 1.1151 - val_accuracy: 0.5805 - val_loss: 1.1103
Epoch 5/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 11s 47ms/step - accuracy: 0.5861 - loss: 1.0937 - val_accuracy: 0.5814 - val_loss: 1.1169
Epoch 6/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 21s 49ms/step - accuracy: 0.5930 - loss: 1.0788 - val_accuracy: 0.5873 - val_loss: 1.0884
Epoch 7/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 21s 49ms/step - accuracy: 0.5931 - loss: 1.0622 - val_accuracy: 0.5897 - val_loss: 1.0949
Epoch 8/50
226/226 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - accuracy: 0.6068 - loss: 1.0384 - 

In [25]:
model_json = model.to_json()
with open("emotiondetector.json", "w") as json_file:
  json_file.write(model_json)
model.save("emotiondetector.h5")

In [26]:
label=['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [29]:
def ef(image):
  img = load_img(image, color_mode = "grayscale")
  feature = np.array(img)
  feature = feature.reshape(1,48,48,1)
  return feature/255.0

In [34]:
image = "images/train/sad/3.jpg"
print("Original Sad Image")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is", pred_label)

Original Sad Image
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
model prediction is sad
